In [1]:
import os
#os.environ["ACCELERATE_MIXED_PRECISION"] = "no"

In [2]:
pip install -U peft transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 12.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 96.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.3 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [3]:
# ============================================================================
# COMPLETE FIXED MEDGEMMA FINE-TUNING NOTEBOOK
# Dataset: alvinl29/medical-vision-llm-dataset-v2
# ============================================================================

# ==========================
# Cell 1: Setup & Installs
# ==========================
!pip install -q transformers datasets accelerate peft trl bitsandbytes safetensors sentencepiece

# Downgrade protobuf if needed
!pip install --upgrade --force-reinstall "protobuf==3.20.3"

import os
WORKDIR = "/content/medgemma_finetune"
os.makedirs(WORKDIR, exist_ok=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.4/517.4 kB 9.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 45.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 whi

In [4]:
# ==========================
# Cell 2: Imports
# ==========================
import gc
import torch
from pathlib import Path
from typing import Any, Dict, List
from PIL import Image

from datasets import load_dataset
from transformers import (
    AutoProcessor,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

print("✓ Imports successful")

2025-12-14 14:09:18.364503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765721358.553330      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765721358.607681      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

✓ Imports successful


In [5]:
# ==========================
# Cell 3: Load Dataset
# ==========================
HF_DATASET_ID = "alvinl29/medical-vision-llm-dataset-v2"
print(f"Loading dataset: {HF_DATASET_ID}")

raw = load_dataset(HF_DATASET_ID)
print("✓ Dataset loaded")
print(f"  - Train: {len(raw['train'])} examples")
print(f"  - Validation: {len(raw['validation'])} examples")


Loading dataset: alvinl29/medical-vision-llm-dataset-v2


README.md:   0%|          | 0.00/412 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/408M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/412M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3035 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/758 [00:00<?, ? examples/s]

✓ Dataset loaded
  - Train: 3035 examples
  - Validation: 758 examples


In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")


In [7]:
import torch
torch.set_default_dtype(torch.float16)

In [9]:
# ==========================
# Cell 4: Login to HuggingFace
# ==========================
from huggingface_hub import notebook_login
notebook_login()

In [10]:
# ==========================\n
# Cell 5: Load Model & Processor (FIXED FOR T4/P100)
# ==========================\n
BASE_MODEL = "google/medgemma-4b-it"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

print(f"Loading model: {BASE_MODEL}")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.float16,  # <--- CRITICAL FIX: Forces all weights to FP16
)

processor = AutoProcessor.from_pretrained(BASE_MODEL)
processor.tokenizer.padding_side = "right"
if processor.tokenizer.pad_token_id is None:
    processor.tokenizer.pad_token_id = processor.tokenizer.eos_token_id

print("✓ Model and processor loaded")

Loading model: google/medgemma-4b-it


config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

✓ Model and processor loaded


In [12]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# THIS IS CRITICAL
model = prepare_model_for_kbit_training(model)

In [14]:
# ==========================
# Cell 6: LoRA Configuration
# ==========================
lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 131,153,920 || all params: 4,431,233,392 || trainable%: 2.9598


In [15]:
# Check at least one LoRA / linear grad dtype later
for n, p in model.named_parameters():
    if p.requires_grad:
        print(n, p.dtype)
        break


base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight torch.float32


In [16]:
# ==========================
# Cell 7: Format Dataset
# ==========================
def format_data_to_messages(example: Dict[str, Any]) -> Dict[str, Any]:
    """Convert dataset format to chat messages format."""
    
    # Build the user prompt
    prompt = (
        "You are a medical vision-language assistant.\n"
        "Analyze the following image and answer the user's question.\n\n"
        f"Image Modality: {example.get('modality', 'Unknown')}\n"
        f"Body Part: {example.get('body_part', 'Unknown')}\n"
        f"Image Description: {example.get('image_description', '')}\n\n"
        f"Question: {example.get('question', '')}\n"
        "Answer:"
    )
    
    # Create messages structure
    example["messages"] = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt},
            ],
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": example.get('answer', '')},
            ],
        },
    ]
    
    return example


print("Formatting datasets...")
train_formatted = raw['train'].map(
    format_data_to_messages,
    remove_columns=[col for col in raw['train'].column_names if col not in ['image', 'messages']]
)

val_formatted = raw['validation'].map(
    format_data_to_messages,
    remove_columns=[col for col in raw['validation'].column_names if col not in ['image', 'messages']]
)

print(f"✓ Train formatted: {len(train_formatted)} examples")
print(f"✓ Val formatted: {len(val_formatted)} examples")


Formatting datasets...


Map:   0%|          | 0/3035 [00:00<?, ? examples/s]

Map:   0%|          | 0/758 [00:00<?, ? examples/s]

✓ Train formatted: 3035 examples
✓ Val formatted: 758 examples


In [17]:
# ==========================
# Cell 8: Collate Function
# ==========================
def collate_fn(examples: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
    """Collate function for batching."""
    texts = []
    images = []
    
    for example in examples:
        # Ensure image is RGB
        img = example["image"]
        if not isinstance(img, Image.Image):
            img = Image.open(img)
        img = img.convert("RGB")
        images.append([img])  # Wrap in list!
        
        # Apply chat template
        text = processor.apply_chat_template(
            example["messages"],
            add_generation_prompt=False,
            tokenize=False,
        ).strip()
        texts.append(text)
    
    # Process batch
    batch = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048,
    )
    
    # Create labels
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    
    # Mask prompt, keep assistant response
    for i, example in enumerate(examples):
        assistant_text = example["messages"][1]["content"][0]["text"]
        # Simple masking approach
        labels[i, :batch["input_ids"].shape[1] - len(assistant_text)] = -100
    
    batch["labels"] = labels
    return batch

print("✓ Collate function defined")


✓ Collate function defined


In [18]:
# ==========================
# Cell 9: Training Arguments (SFTConfig)
# ==========================
from trl import SFTConfig

training_args = SFTConfig(
    output_dir=WORKDIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    fp16=False,
    bf16=False,
    num_train_epochs=1,  # Start with 1 epoch
    logging_steps=50,
    save_steps=500,
    save_total_limit=5,
    eval_strategy="steps",
    eval_steps=500,
    learning_rate=1e-5,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    dataloader_num_workers=0,
    dataloader_pin_memory=True,
    label_names=["labels"],
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_kwargs={"skip_prepare_dataset": True},
    optim="adamw_torch_fused",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    max_grad_norm=None,
)

print("✓ Training arguments configured")

✓ Training arguments configured


In [19]:
# ==========================
# Cell 10: Create Trainer
# ==========================
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_formatted,
    eval_dataset=val_formatted,
    processing_class=processor,
    data_collator=collate_fn,
)

print("\n" + "="*60)
print("✅ READY TO TRAIN!")
print("="*60)
print(f"\nDataset: {len(train_formatted)} train, {len(val_formatted)} val")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total steps: ~{len(train_formatted) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")


✅ READY TO TRAIN!

Dataset: 3035 train, 758 val
Effective batch size: 8
Total steps: ~379


In [20]:
print("Model dtype:", next(model.parameters()).dtype)


Model dtype: torch.float32


In [21]:
print(trainer.args.fp16, trainer.args.bf16)


False False


In [22]:
print("fp16:", trainer.args.fp16)
print("bf16:", trainer.args.bf16)
print("use_amp:", trainer.accelerator.use_fp16)


fp16: False
bf16: False


AttributeError: 'Accelerator' object has no attribute 'use_fp16'

In [23]:
print(next(model.parameters()).dtype)  # float16
print(trainer.args.fp16, trainer.args.bf16)  # True False


torch.float32
False False


In [24]:
import torch

bf16_params = []
for name, p in model.named_parameters():
    if p.requires_grad and p.dtype == torch.bfloat16:
        bf16_params.append(name)

print("BF16 trainable params:", bf16_params[:10])
print("Count:", len(bf16_params))


BF16 trainable params: ['base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.1.self_attn.k_proj.

In [25]:
bf16_params = [
    n for n, p in model.named_parameters()
    if p.requires_grad and p.dtype == torch.bfloat16
]
print(len(bf16_params))  # must be 0


638


In [26]:
import torch
from collections import Counter

def audit_dtypes(model, trainer=None):
    print("\n===== GLOBAL SETTINGS =====")
    print("CUDA available:", torch.cuda.is_available())
    print("Default dtype:", torch.get_default_dtype())
    print("AMP autocast enabled:", torch.is_autocast_enabled())

    if trainer is not None:
        print("\n===== TRAINER / ACCELERATE =====")
        print("fp16:", trainer.args.fp16)
        print("bf16:", trainer.args.bf16)
        try:
            print("accelerate mixed_precision:", trainer.accelerator.state.mixed_precision)
        except Exception:
            print("accelerate mixed_precision: <unknown>")

    print("\n===== PARAMETER DTYPES (SUMMARY) =====")
    param_dtypes = Counter(p.dtype for p in model.parameters())
    for k, v in param_dtypes.items():
        print(f"{k}: {v}")

    print("\n===== TRAINABLE PARAMETER DTYPES =====")
    trainable = Counter(p.dtype for p in model.parameters() if p.requires_grad)
    for k, v in trainable.items():
        print(f"{k}: {v}")

    print("\n===== LoRA PARAMETER DTYPES =====")
    lora_dtypes = Counter()
    for name, p in model.named_parameters():
        if "lora" in name.lower():
            lora_dtypes[p.dtype] += 1
    if lora_dtypes:
        for k, v in lora_dtypes.items():
            print(f"{k}: {v}")
    else:
        print("No LoRA parameters found")

    print("\n===== BF16 TRAINABLE PARAMETERS (NAMES) =====")
    bf16_names = [
        name for name, p in model.named_parameters()
        if p.requires_grad and p.dtype == torch.bfloat16
    ]
    if bf16_names:
        print("Count:", len(bf16_names))
        print("First 10:")
        for n in bf16_names[:10]:
            print(" ", n)
    else:
        print("None")

    print("\n===== FP16 TRAINABLE PARAMETERS (NAMES) =====")
    fp16_names = [
        name for name, p in model.named_parameters()
        if p.requires_grad and p.dtype == torch.float16
    ]
    if fp16_names:
        print("Count:", len(fp16_names))
        print("First 10:")
        for n in fp16_names[:10]:
            print(" ", n)
    else:
        print("None")

    print("\n===== FP32 TRAINABLE PARAMETERS (NAMES) =====")
    fp32_names = [
        name for name, p in model.named_parameters()
        if p.requires_grad and p.dtype == torch.float32
    ]
    if fp32_names:
        print("Count:", len(fp32_names))
        print("First 10:")
        for n in fp32_names[:10]:
            print(" ", n)
    else:
        print("None")

    print("\n===== DONE =====\n")


# USAGE:
audit_dtypes(model, trainer)



===== GLOBAL SETTINGS =====
CUDA available: True
Default dtype: torch.float16
AMP autocast enabled: False

===== TRAINER / ACCELERATE =====
fp16: False
bf16: False
accelerate mixed_precision: no

===== PARAMETER DTYPES (SUMMARY) =====
torch.float32: 483
torch.uint8: 400
torch.bfloat16: 638

===== TRAINABLE PARAMETER DTYPES =====
torch.bfloat16: 638

===== LoRA PARAMETER DTYPES =====
torch.bfloat16: 638

===== BF16 TRAINABLE PARAMETERS (NAMES) =====
Count: 638
First 10:
  base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight
  base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.lora_B.default.weight
  base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight
  base_model.model.base_model.model.model.vision_tower.vision_model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight
  base_model.model.base_model

In [27]:
# ==========================
# Cell 11: Train!
# ==========================
print("\n🚀 Starting training...")
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.



🚀 Starting training...


Step,Training Loss,Validation Loss


TrainOutput(global_step=380, training_loss=1.3419783140483657, metrics={'train_runtime': 29723.6789, 'train_samples_per_second': 0.102, 'train_steps_per_second': 0.013, 'total_flos': 2.462890294676928e+16, 'train_loss': 1.3419783140483657, 'entropy': 3.486027053061952, 'num_tokens': 1093084.0, 'mean_token_accuracy': 0.842724609375, 'epoch': 1.0})

In [29]:
# ==========================
# Cell 12: Save Adapter
# ==========================
WORKDIR1="/kaggle/working/"
ADAPTER_DIR = os.path.join(WORKDIR1, "final_adapter")
os.makedirs(ADAPTER_DIR, exist_ok=True)

print(f"\n💾 Saving adapter to {ADAPTER_DIR}...")
model.save_pretrained(ADAPTER_DIR)
processor.save_pretrained(ADAPTER_DIR)

print("✅ Training complete!")
print(f"📁 Adapter saved to: {ADAPTER_DIR}")

# Optional: Create zip for download
import shutil
zip_path = ADAPTER_DIR + ".zip"
shutil.make_archive(ADAPTER_DIR, 'zip', ADAPTER_DIR)
print(f"📦 Zip created: {zip_path}")


💾 Saving adapter to /kaggle/working/final_adapter...
✅ Training complete!
📁 Adapter saved to: /kaggle/working/final_adapter
📦 Zip created: /kaggle/working/final_adapter.zip


In [30]:
!ls /kaggle/working


final_adapter  final_adapter.zip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
!zip -r /kaggle/working/lora_adapter.zip /kaggle/working/lora_adapter



	zip warning: name not matched: /kaggle/working/lora_adapter

zip error: Nothing to do! (try: zip -r /kaggle/working/lora_adapter.zip . -i /kaggle/working/lora_adapter)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
!pip install -q PyDrive


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
